In [2]:
%pip install llama-index-llms-litellm

%pip install llama-index


  Using cached aiohttp-3.11.18-cp312-cp312-win_amd64.whl.metadata (8.0 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached openai-1.75.0-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic-2.11.4-py3-none-any.whl.metadata (66 kB)
  Using cached tiktoken-0.9.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.24.0-cp312-cp312-win_amd64.whl.metadata (4.2 kB)
  Using cache

In [3]:
%pip install llama-index-graph-stores-neo4j



   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ------------------- ------------------- 1/2 [llama-index-graph-stores-neo4j]
   --------------------------------------- 2/2 [llama-index-graph-stores-neo4j]

Note: you may need to restart the kernel to use updated packages.


# Knowledge Graph with Neo4j and LlamaIndex

This notebook creates a knowledge graph from documents using LlamaIndex and Neo4j.

In [ ]:
import os
from llama_index.llms.litellm import LiteLLM
from llama_index.embeddings.openai import OpenAIEmbedding



In [ ]:

import nest_asyncio
nest_asyncio.apply()

In [2]:
import dotenv
dotenv.load_dotenv()

True

In [15]:
llm = LiteLLM(model="gemini/gemini-2.0-flash-lite",api_key=os.getenv("GOOGLE_API_KEY"))
embedding = OpenAIEmbedding(model="text-embedding-ada-002", api_key=os.getenv("OPENAI_API_KEY"))

In [18]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [ ]:

# # For local Neo4j instance if you prefer that
username = "neo4j"
password = "password"  # default password for local Neo4j installation
url = "bolt://localhost:7687"  # standard localhost URL for Neo4j

In [ ]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import PropertyGraphIndex

# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    url=url,
    username=username,
    password=password)

In [19]:
documents= SimpleDirectoryReader("data").load_data()

In [22]:
# Create and build the knowledge graph index
index = KnowledgeGraphIndex.from_documents(
    documents,
    property_graph_store=graph_store,
    max_triplets_per_chunk=10,
    show_progress=True,
    include_embeddings=True,
    llm=llm,
    embed_model=embedding,
)

Processing nodes:   0%|          | 0/37 [00:00<?, ?it/s]

Processing nodes:   3%|▎         | 1/37 [00:04<02:32,  4.24s/it]

Processing nodes:   5%|▌         | 2/37 [00:08<02:32,  4.36s/it]

Processing nodes:   8%|▊         | 3/37 [00:12<02:15,  3.98s/it]

Processing nodes:  11%|█         | 4/37 [00:15<02:02,  3.70s/it]

Processing nodes:  14%|█▎        | 5/37 [00:19<01:56,  3.64s/it]

Processing nodes:  16%|█▌        | 6/37 [00:22<01:53,  3.67s/it]

Processing nodes:  19%|█▉        | 7/37 [00:26<01:49,  3.66s/it]

Processing nodes:  22%|██▏       | 8/37 [00:29<01:38,  3.41s/it]

Processing nodes:  24%|██▍       | 9/37 [00:32<01:30,  3.21s/it]

Processing nodes:  27%|██▋       | 10/37 [00:35<01:29,  3.30s/it]

Processing nodes:  30%|██▉       | 11/37 [00:37<01:19,  3.04s/it]

Processing nodes:  32%|███▏      | 12/37 [00:40<01:13,  2.93s/it]

Processing nodes:  35%|███▌      | 13/37 [00:43<01:11,  2.99s/it]

Processing nodes:  38%|███▊      | 14/37 [00:46<01:08,  2.96s/it]

Processing no

# Querying and Retrieval


## Querying

In [24]:
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

nodes = retriever.retrieve("ในสมการ (2.5) และ (2.6) ของ ACI มีเงื่อนไขการใช้สมการอย่างไร และตัวแปร h/r มีความสำคัญอย่างไร?")

for node in nodes:
    print(node.text)


The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('การใช้เทคโนโลยี', 'มีประสิทธิภาพ', 'สูงกว่า')
['ตัวแปร', 'ประกอบด้วย', 'อัตราส่วนเหล็กเสริม']
['ตัวแปร', 'ประกอบด้วย', 'อัตราส่วนความชะลูด']
('มาตรฐาน aci', 'พิจารณา', 'ปูนกรอก')
['ตัวแปร', 'ใช้', 'ศึกษาแรงกดอัด']
['สมการ', 'เหมาะสม', 'ออกแบบ']
['ความสำคัญ', 'Is', 'ปัญหา']
['สมการ', 'ไม่มี', 'พจน์']


C:\Users\cepheusn22\AppData\Local\Temp\ipykernel_17700\767407642.py:5: RuntimeWarning: coroutine 'Dispatcher.span.<locals>.async_wrapper' was never awaited
  nodes = retriever.retrieve("ในสมการ (2.5) และ (2.6) ของ ACI มีเงื่อนไขการใช้สมการอย่างไร และตัวแปร h/r มีความสำคัญอย่างไร?")


## Retrieval

In [25]:
query_engine = index.as_query_engine(include_text=True)

response = query_engine.query("ในสมการ (2.5) และ (2.6) ของ ACI มีเงื่อนไขการใช้สมการอย่างไร และตัวแปร h/r มีความสำคัญอย่างไร?")

print(str(response))

ในสมการ (2.5) และ (2.6) ของ ACI มีเงื่อนไขการใช้เช่นเมื่อ h/r มีค่าน้อยกว่าหรือเท่ากับ 99 จะใช้สมการ (2.5) และเมื่อ h/r มีค่ามากกว่า 99 จะใช้สมการ (2.6) สำหรับการคำนวณ ตัวแปร h/r ใช้ในการกำหนดว่าจะใช้สมการใดในการคำนวณ โดยมีความสำคัญในการกำหนดวิธีการคำนวณแรงบรรทุกที่กระทำบนผนังของคอนกรีตบล็อกเสริมเหล็กในแนวแกนในแต่ละเงื่อนไขของ h/r ที่กำหนดในสมการ (2.5) และ (2.6) ที่มีผลต่อการคำนวณแรงบรรทุกที่ผนังสามารถรับได้ในแต่ละกรณี.


# Loading from an existing Graph




In [ ]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import PropertyGraphIndex


graph_store = Neo4jPropertyGraphStore(
    url=url,
    username=username,
    password=password,
    # use a different graph name to avoid collision with the previous index
)
index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    llm=llm,
    embed_model=embedding,
)

## Insert New

In [ ]:

from llama_index.core import Document

document = Document(text="LlamaIndex is great!")

index.insert(document)

In [31]:
nodes = index.as_retriever(include_text=True).retrieve("ในสมการ (2.5) และ (2.6) ของ ACI มีเงื่อนไขการใช้สมการอย่างไร และตัวแปร h/r มีความสำคัญอย่างไร?")

print(nodes)

[]
